*Copyright 2019 StarkWare Industries Ltd.<br> Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.starkware.co/open-source-license/ <br> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.*

# Part 3: FRI承诺

- [Video Lecture (youtube)](https://www.youtube.com/watch?v=gd1NbKUOJwA)
- [Slides (PDF)](https://starkware.co/wp-content/uploads/2021/12/STARK101-Part3.pdf)

### 加载上一章节内容
运行下面代码来加载相关内容。耐心等待，他们需要一点时间运行。

In [24]:
from channel import Channel
from field import FieldElement
from merkle import MerkleTree
from polynomial import interpolate_poly, Polynomial 
from tutorial_sessions import part1, part2

cp, cp_eval, cp_merkle, channel, eval_domain = part2()
print("Success")

Success


## FRI 折叠

这部分中，我们的目标是构造FRI层并对他们进行承诺。

为了获取每一层，我们需要做到：
1. 生成当前层的域（从上一层的域中获取）
2. 生成当前层的多项式（从上一层的多项式和域中获取）
3. 在所述域上评估所述多项式 - **这就是下一个FRI层**

### 域生成-生成当前域

第一个FRI域就是你在第一部分中生成的`eval_domain`，即阶为8192的群的陪集。每个后续的FRI域都是通过取前一个FRI域的前一半（丢弃后一半），然后对每个元素进行平方得到的。


专业一点来说，我们可以通过以下公式来生成：

$$w, w\cdot h, w\cdot h^2, ..., w\cdot h^{8191}$$

因此，下一层将是：

$$w^2, (w\cdot h)^2, (w\cdot h^2)^2, ..., (w\cdot h^{4095})^2$$

请注意，对`eval_domain`的第二半部分的平方与对`eval_domain`的第一个半部分的平方是相同的。这对于下一层也同样成立。

例如：

In [17]:
print(eval_domain[100] ** 2)
half_domain_size = len(eval_domain) // 2
print(eval_domain[half_domain_size + 100] ** 2)

-373161870
8192
-373161870


类似的，第三层的域将是：

$$w^4, (w\cdot h)^4, (w\cdot h^2)^4, ..., (w\cdot h^{2047})^4$$

以此类推

编写一个函数`next_fri_domain`，它接受上一个域作为输入，并输出下一个域。

In [3]:
def next_fri_domain(fri_domain):
    # Fix this.
    return [x for x in fri_domain[:len(fri_domain) // 2]]

答案:

In [4]:
def next_fri_domain(fri_domain):
    return [x ** 2 for x in fri_domain[:len(fri_domain) // 2]]

运行测试: 

In [5]:
# Test against a precomputed hash.
from hashlib import sha256
next_domain = next_fri_domain(eval_domain)
assert '5446c90d6ed23ea961513d4ae38fc6585f6614a3d392cb087e837754bfd32797' == sha256(','.join([str(i) for i in next_domain]).encode()).hexdigest()
print('Success!')

Success!


### 多项式生成-生成当前层的多项式(原文为FRI折叠运算符)
第一个FRI多项式就是组合多项式，即 `cp` 。

后续每一个FRI多项式都是通过以下步骤得到的：

1. 获取一个随机域元素 $\beta$ (通过调用 `Channel.receive_random_field_element` 来获取)。
2. 将前一个多项式的奇数系数乘以 $\beta$ 。
3. 将相邻的偶数和奇数系数相加。


专业一点来说，假设第k个多项式的度 $ < m $ (其中 $m$ 是2的幂)

$$p_{k}(x) := \sum _{i=0} ^{m-1} c_i x^i$$

那么第k+1个多项式，其度数为 $< \frac m 2 $ ，如下：

$$p_{k+1}(x) := \sum _{i=0} ^{  m / 2 - 1 } (c_{2i} + \beta \cdot c_{2i + 1}) x^i$$

编写一个函数 `next_fri_polynomial` ，它接受一个多项式和一个域元素 (我们称之为 $\beta$ )，并返回 "折叠" 后的下一个多项式。

注意：
1. `Polynomial.poly` 包含一个多项式系数的列表，常数项，最后最高次项，所以如果 $x^i$ 的系数是 $u$ ，那么 `p.poly[i] == u`。*
2. `Polynomial` 的默认构造函数接受系数列表作为参数。因此，可以通过调用 Polynomial(l) 从系数列表 l 实例化一个多项式。

In [6]:
def next_fri_polynomial(poly, beta):
    odd_coefficients = poly.poly[1::2] # No need to fix this line.
    even_coefficients = poly.poly[::2] # No need to fix this line either.
    odd = 'YOUR_CODE_HERE'
    even = 'YOUR_CODE_HERE'
    return 'YOUR_CODE_HERE'

答案:

In [7]:
def next_fri_polynomial(poly,  beta):
    odd_coefficients = poly.poly[1::2]
    even_coefficients = poly.poly[::2]
    odd = beta * Polynomial(odd_coefficients)
    even = Polynomial(even_coefficients)
    return odd + even

运行测试:

In [8]:
next_p = next_fri_polynomial(cp, FieldElement(987654321))
assert '6bff4c35e1aa9693f9ceb1599b6a484d7636612be65990e726e52a32452c2154' == sha256(','.join([str(i) for i in next_p.poly]).encode()).hexdigest()
print('Success!')

Success!


### 整合以获得下一个FRI层-在所述域上计算所述多项式

编写一个函数 `next_fri_layer` ，它接受一个多项式，一个域，和一个域元素 ($\beta$)，并返回下一个多项式，下一个域，以及这个新多项式在新域上的求值结果。

In [9]:
def next_fri_layer(poly, domain, beta):
    next_poly = 'YOUR_CODE_HERE'
    next_domain = 'YOUR_CODE_HERE'
    next_layer = 'YOUR_CODE_HERE'
    return next_poly, next_domain, next_layer

答案:

In [10]:
def next_fri_layer(poly, domain, beta):
    next_poly = next_fri_polynomial(poly, beta)
    next_domain = next_fri_domain(domain)
    next_layer = [next_poly(x) for x in next_domain]
    return next_poly, next_domain, next_layer

运行测试:

In [11]:
test_poly = Polynomial([FieldElement(2), FieldElement(3), FieldElement(0), FieldElement(1)])
test_domain = [FieldElement(3), FieldElement(5)]
beta = FieldElement(7)
next_p, next_d, next_l = next_fri_layer(test_poly, test_domain, beta)
assert next_p.poly == [FieldElement(23), FieldElement(7)]
assert next_d == [FieldElement(9)]
assert next_l == [FieldElement(86)]
print('Success!')

Success!


## 生成FRI承诺

我们现在已经开发了编写 `FriCommit` 方法所需要的工具，该方法包含主要的FRI承诺循环。


它接受以下5个参数:

1. 组合多项式，即第一个FRI多项式，即 `cp`。
2. 8192阶陪集，也是第一个FRI域，即 `eval_domain`。
3. 前者在后者上的求值，也是第一个FRI层，即 `cp_eval`。
4. 从这些求值构造的第一个默克尔树（我们将为每个FRI层构造一个），即 `cp_merkle`。
5. 通道对象，即 `channel`。


相应地，该方法返回4个列表：
1. FRI多项式。
2. FRI域。
3. FRI层。
4. FRI Merkle 树。

该方法包含一个循环，每次迭代中，我们使用每个列表最后一个元素来拓展这四个列表。

当最后一个FRI多项式是常数时(也就是度为0时)，循环应该停止。然后，它应该将这个常数（即多项式的常数项）发送到通道上。

`Channel` 类只支持发送字符串，因此在发送之前，请确保将要发送的任何内容转换为字符串。

In [12]:
# 根据介绍修改代码（没有注释的行不用改）
def FriCommit(cp, domain, cp_eval, cp_merkle, channel):
    fri_polys = [cp]
    fri_domains = [domain]
    fri_layers = [cp_eval]
    fri_merkles = [cp_merkle]
    while 'YOUR_CODE_HERE': # 替换成正确的停止条件
        beta = 'YOUR_CODE_HERE' #  修改成从通道中获取随机元素
        next_poly, next_domain, next_layer = 'YOUR_CODE_HERE' #  修改成获取下一个FRI多项式、域和层
        fri_polys.append(next_poly)
        fri_domains.append(next_domain)
        fri_layers.append(next_layer)
        fri_merkles.append('YOUR_CODE_HERE') #  修改成构造正确的默克尔树
        channel.send('YOUR_CODE_HERE') #  修改成发送正确的承诺
    channel.send('YOUR_CODE_HERE') #  修改成发送最后一层的自由项
    return fri_polys, fri_domains, fri_layers, fri_merkles

答案:

In [34]:
def FriCommit(cp, domain, cp_eval, cp_merkle, channel):    
    fri_polys = [cp]
    fri_domains = [domain]
    fri_layers = [cp_eval]
    fri_merkles = [cp_merkle]
    while fri_polys[-1].degree() > 0:
        beta = channel.receive_random_field_element()
        next_poly, next_domain, next_layer = next_fri_layer(fri_polys[-1], fri_domains[-1], beta)
        fri_polys.append(next_poly)
        fri_domains.append(next_domain)
        fri_layers.append(next_layer)
        fri_merkles.append(MerkleTree(next_layer))
        channel.send(fri_merkles[-1].root)   
    channel.send(str(fri_polys[-1].poly[0]))
    return fri_polys, fri_domains, fri_layers, fri_merkles

运行测试:

In [14]:
test_channel = Channel()
fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, test_channel)
assert len(fri_layers) == 11, f'Expected number of FRI layers is 11, whereas it is actually {len(fri_layers)}.'
assert len(fri_layers[-1]) == 8, f'Expected last layer to contain exactly 8 elements, it contains {len(fri_layers[-1])}.'
assert all([x == FieldElement(-1138734538) for x in fri_layers[-1]]), f'Expected last layer to be constant.'
assert fri_polys[-1].degree() == 0, 'Expacted last polynomial to be constant (degree 0).'
assert fri_merkles[-1].root == '1c033312a4df82248bda518b319479c22ea87bd6e15a150db400eeff653ee2ee', 'Last layer Merkle root is wrong.'
assert test_channel.state == '61452c72d8f4279b86fa49e9fb0fdef0246b396a4230a2bfb24e2d5d6bf79c2e', 'The channel state is not as expected.'
print('Success!')

Success!


运行下面代码，并且打印通道中的证明内容

In [35]:
fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)
print(channel.proof) 

['send:6c266a104eeaceae93c14ad799ce595ec8c2764359d7ad1b4b7c57a4da52be04', 'receive_random_field_element:2948900820', 'receive_random_field_element:1859037345', 'receive_random_field_element:2654806830', 'send:61f7d8283e244d391a483c420776e351fcfdbce525a698461a8307a1345b5652', 'receive_random_field_element:394024765', 'receive_random_field_element:902225401', 'receive_random_field_element:2586476798', 'send:dc87249e62b35141c51fe73c541e2ac4929cbcedf012ad9c2c93be0f9f72b778', 'receive_random_field_element:1751808743', 'send:a4ca0211faeda0bc13fdee29fb0f005656b4a6f5952fbc0c218541e4b9b52093', 'receive_random_field_element:2983935505', 'send:b17ebd83c25ee2d8f4a52265d33e3954a9734a7814c0dc56c732ba51e83ebaf8', 'receive_random_field_element:3100871748', 'send:cfd52d4f43c446a9ad632f52872498262d60b73c477d247b26782397b6b38914', 'receive_random_field_element:2795361144', 'send:25973e983015bf2318a35219d53b9bba93ce6f7c3046e27eb1d79c21bee8e6c3', 'receive_random_field_element:2859576984', 'send:45841da2806